In [3]:
import numpy as np
import numpy.ma as ma
import os
import astropy.units as u
from astropy.io import ascii
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.stats import sigma_clipped_stats, sigma_clip
import subprocess
from astropy.io.ascii import write



In [4]:
cd /home/poorvi/Desktop/ph556_astro/project/

/home/poorvi/Desktop/ph556_astro/project


In [5]:
from astropy.io import fits
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd
from math import modf
import numpy as np

data_dir='data'
date_folders = [
    f.path for f in os.scandir(data_dir) if not f.is_file()]
date_folders.sort()


images=[]
img_ind=[]
date_ind=[]

for i in range(len(date_folders)):
    date=date_folders[i]
    new_images = [f.path for f in os.scandir(date) if f.is_file() and f.path.endswith(('.fits'))]
    l=len(new_images)
    date_ind=np.append(date_ind, np.repeat(i,l))
    images.append(new_images)
    

all_images = [item for sublist in images for item in sublist]
print(len(all_images))

df=pd.DataFrame({'img_index': np.arange(len(all_images)), 'images': all_images , 'date_index': date_ind})
from dateutil import parser
P=0.736941*24*60*60
times=[]
for img in all_images:
    f = fits.open(img)
    data = f[0].data 
    header = f[0].header
    date=header['DATE-OBS'][:10]
    time=header['DATE-OBS'][11:]
    date_time_str=date + ' '+ time
    t= parser.parse(date_time_str)
    times.append(t)

times.sort()
phase=[modf(((ti-times[0]).total_seconds())/P)[0] for ti in times ]

df['time']=times
df['phase']=phase
print(df)

35
    img_index                                             images  date_index  \
0           0             data/20210319/UZCom-2-RA.wcs.proc.fits         0.0   
1           1             data/20210319/UZcom-3-RA.wcs.proc.fits         0.0   
2           2             data/20210319/UZcom-4-RA.wcs.proc.fits         0.0   
3           3             data/20210319/UZCom-1-RA.wcs.proc.fits         0.0   
4           4             data/20210322/UZCom-5-RA.wcs.proc.fits         1.0   
5           5             data/20210322/UZCom-2-RA.wcs.proc.fits         1.0   
6           6             data/20210322/UZCom-7-RA.wcs.proc.fits         1.0   
7           7             data/20210322/UZCom-3-RA.wcs.proc.fits         1.0   
8           8             data/20210322/UZCom-1-RA.wcs.proc.fits         1.0   
9           9            data/20210322/UZCom-11-RA.wcs.proc.fits         1.0   
10         10            data/20210322/UZCom-12-RA.wcs.proc.fits         1.0   
11         11             data/202103

In [6]:
def get_table_from_ldac(filename, frame=1):
    """
    Load an astropy table from a fits_ldac by frame (Since the ldac format has column 
    info for odd tables, giving it twce as many tables as a regular fits BinTableHDU,
    match the frame of a table to its corresponding frame in the ldac file).
    
    Parameters
    ----------
    filename: str
        Name of the file to open
    frame: int
        Number of the frame in a regular fits file
    """
    from astropy.table import Table
    if frame>0:
        frame = frame*2
    tbl = Table.read(filename, hdu=frame)
    return tbl

In [8]:
from astroquery.vizier import Vizier
from astropy import units as u
from photutils import SkyCircularAperture, SkyCircularAnnulus, aperture_photometry
import warnings
warnings.filterwarnings('ignore')

def calculate_mag(imageName):
    f = fits.open(imageName)
    data = f[0].data 
    header = f[0].header

    mean, median, sigma = sigma_clipped_stats(data) 
    w = WCS(header)
    [raImage, decImage] = w.all_pix2world(data.shape[0]/2, data.shape[1]/2, 1)
    boxsize = 30
    maxmag = 18
    catNum = 'II/349'

    try:
        v = Vizier(columns=['*'], column_filters={"gmag":"<<%.2f"%maxmag, "Nd":">>6", "e_gmag":"<<1.086/3"}, row_limit=-1)
        Q = v.query_region(SkyCoord(ra = raImage, dec = decImage, unit = (u.deg, u.deg)), radius = str(boxsize)+'m', catalog=catNum, cache=False)
    except:
        print('I cannnot reach the Vizier database. Is the internet working?')
        
    ps1_imCoords = w.all_world2pix(Q[0]['RAJ2000'], Q[0]['DEJ2000'], 1)
    good_cat_stars = Q[0][np.where((ps1_imCoords[0] < 3500) & (ps1_imCoords[1] < 3500))]
    ps1_imCoords = w.all_world2pix(good_cat_stars['RAJ2000'],good_cat_stars['DEJ2000'], 1)
    configFile = 'photomCat.sex'
    catalogName = imageName+'.cat'
    paramName = 'photomCat.param'
    
    try:
        command = 'source-extractor -c %s %s -CATALOG_NAME %s -PARAMETERS_NAME %s' % (configFile, imageName, catalogName, paramName)
        rval = subprocess.run(command.split(), check=True)
    except subprocess.CalledProcessError as err:
        print('Could not run sextractor with exit error %s'%err)
        
    sourceTable = get_table_from_ldac(catalogName)
    cleanSources = sourceTable[(sourceTable['FLAGS']==0) & (sourceTable['FWHM_WORLD'] < 2) & (sourceTable['XWIN_IMAGE']<3500) & (sourceTable['YWIN_IMAGE']<3500)]
    sourceCatCoords = SkyCoord(ra=cleanSources['ALPHAWIN_J2000'], dec=cleanSources['DELTAWIN_J2000'], frame='icrs', unit='degree')
    ps1CatCoords = SkyCoord(ra=good_cat_stars['RAJ2000'], dec=good_cat_stars['DEJ2000'], frame='icrs', unit='degree')
    photoDistThresh = 0.6
    idx_image, idx_ps1, d2d, d3d = ps1CatCoords.search_around_sky(sourceCatCoords, photoDistThresh*u.arcsec)
 
    aperture_diameter = np.arange(10, 11)

    magDiff = np.ma.zeros((len(aperture_diameter), len(idx_image)))
    for j in range(len(aperture_diameter)):
        magDiff[j] = sigma_clip(cleanSources['MAG_APER'][:,9][idx_image] - cleanSources['MAG_APER'][:,j][idx_image])
    zeroPoints = []
    for i in range(len(aperture_diameter)):
        offsets = ma.array(good_cat_stars['gmag'][idx_ps1] - cleanSources['MAG_APER'][:,i][idx_image])
        zero_mean, zero_med, zero_std = sigma_clipped_stats(offsets)
        zeroDict = {'diameter': aperture_diameter[i], 'zp_mean': zero_mean, 'zp_median': zero_med, 'zp_std': zero_std}
        zeroPoints.append(zeroDict)
    ra = 198.11145334
    dec = 30.35444063
    position = SkyCoord(ra = ra, dec = dec, unit = u.deg, frame = 'icrs')

    aperture_radii = np.array([10.0]) / 2 #Aperutre radii in pixels 
    apertures = [SkyCircularAperture(position, r = r * u.pix) for r in aperture_radii]
    pix_apertures = [a.to_pixel(w) for a in apertures]
    phot_table = aperture_photometry(data, pix_apertures)
    for col in phot_table.colnames:
        phot_table[col].info.format = '%.8g'  
    anuRadius = 15.0
    anuWidth = 5.0
    annulus_aperture = SkyCircularAnnulus(position, r_in = anuRadius * u.pix, r_out = (anuRadius + anuWidth) * u.pix)
    pix_annulus_aperture = annulus_aperture.to_pixel(w)
    annulus_phot_table = aperture_photometry(data, pix_annulus_aperture)
    for col in annulus_phot_table.colnames:
            annulus_phot_table[col].info.format = '%.8g' 
    bkg_mean = annulus_phot_table['aperture_sum'] / pix_annulus_aperture.area
    
    for i in range(len(pix_apertures)):
        bkg_flux = bkg_mean * pix_apertures[i].area
        source_flux = phot_table['aperture_sum_%d'%i] - bkg_flux
        source_mag = zeroPoints[i]['zp_median'] - 2.5 * np.log10(source_flux)
    sourceCatCoords2 = SkyCoord(ra=[198.11145334], dec=[30.35444063], frame='icrs', unit='degree')
    ps1CatCoords2 = SkyCoord(ra=cleanSources['ALPHAWIN_J2000'], dec=cleanSources['DELTAWIN_J2000'], frame='icrs', unit='degree')
    photoDistThresh2 = 2
    idx_image2, idx_ps12, d2d2, d3d2 = ps1CatCoords2.search_around_sky(sourceCatCoords2, photoDistThresh2*u.arcsec)
    magn=cleanSources["MAG_APER"][int(idx_ps12), 0]+zero_med
    magn_error=cleanSources["MAGERR_APER"][int(idx_ps12), 0]+zero_std
    return magn, magn_error

In [ ]:
magns=[]
magns_error=[]
for img in all_images:
    magn, error= calculate_mag(img)
    magns.append(magn)
    magns_error.append(error)

df['Magnitude']=magns
df['Magnitude error']=magns_error
print(df)


In [42]:
df.to_csv('info.csv')

In [ ]:
from  matplotlib import pyplot
import seaborn
seaborn.set(style='ticks')

np.random.seed(0)

dates_pos=[0,1,2,3,4]
fg = seaborn.FacetGrid(data=df, hue='date_index', hue_order=dates_pos, aspect=5)
fg.map(pyplot.scatter, 'time', 'Magnitude').add_legend()

In [ ]:
from  matplotlib import pyplot
import seaborn
seaborn.set(style='ticks')

np.random.seed(0)

dates_pos=[0,1,2,3,4]
fg = seaborn.FacetGrid(data=df, hue='date_index', hue_order=dates_pos, aspect=2)
fg.map(pyplot.scatter, 'phase', 'Magnitude').add_legend()